# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Import cities file into the cities_pd DataFrame

# Read in the csv data from the csv exported in part I:
# cities_df = pd.read_csv('../resources/weather_output.csv')[:10]

weather_df = pd.read_csv('resources/weather_output.csv')
weather_df.head()

,Unnamed: 0,index,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,0,perth,-31.9333,115.8333,60.78,62.0,0.0,14.83,AU,1620836410
1,1,1,samarai,-10.6167,150.6667,80.83,82.0,57.0,6.71,PG,1620836479
2,2,2,avarua,-21.2078,-159.7750,69.69,100.0,90.0,3.44,CK,1620836433
3,3,3,rikitea,-23.1203,-134.9692,77.70,77.0,21.0,21.00,PF,1620836481
4,4,4,cherskiy,68.7500,161.3000,8.31,98.0,97.0,4.12,RU,1620836483


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Import dependencies

import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)


In [5]:
# Create a heat map that displays the humidity for every city from Part I.

In [6]:
# Creating heat map

fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(weather_df[["Lat", "Lng"]])
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
max_humidity = weather_df["Humidity"].max() 
max_humidity

100.0

In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
#fig = gmaps.figure(map_type='HYBRID')
heat_layer = gmaps.heatmap_layer(weather_df[["Lat", "Lng"]], weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(weather_df[["Lat", "Lng"]])
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_weather_df = weather_df.loc[(weather_df['Max_Temp'] < 80) & (weather_df['Max_Temp'] > 70) & (weather_df['Wind_Speed'] < 10) & (weather_df['Cloudiness'] == 0)]
ideal_weather_df


,Unnamed: 0,index,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
143,143,155,kumluca,36.3703,30.2869,71.85,63.0,0.0,4.59,TR,1620836658
235,235,255,inhambane,-23.8650,35.3833,73.29,73.0,0.0,0.00,MZ,1620836774
287,287,312,maltahohe,-24.8333,16.9833,73.96,17.0,0.0,8.68,NaN,1620836840
370,370,400,saurimo,-9.6608,20.3916,71.82,57.0,0.0,3.44,AO,1620836883
498,498,535,bukama,-9.2000,25.8500,70.48,60.0,0.0,2.59,CD,1620837108
521,521,560,richards bay,-28.7830,32.0377,70.27,86.0,0.0,8.34,ZA,1620836972


In [12]:
ideal_weather_df = ideal_weather_df.dropna()

In [13]:
ideal_weather_df.head()

,Unnamed: 0,index,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
143,143,155,kumluca,36.3703,30.2869,71.85,63.0,0.0,4.59,TR,1620836658
235,235,255,inhambane,-23.8650,35.3833,73.29,73.0,0.0,0.00,MZ,1620836774
370,370,400,saurimo,-9.6608,20.3916,71.82,57.0,0.0,3.44,AO,1620836883
498,498,535,bukama,-9.2000,25.8500,70.48,60.0,0.0,2.59,CD,1620837108
521,521,560,richards bay,-28.7830,32.0377,70.27,86.0,0.0,8.34,ZA,1620836972


In [14]:
ideal_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 143 to 521
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5 non-null      int64  
 1   index       5 non-null      int64  
 2   City        5 non-null      object 
 3   Lat         5 non-null      float64
 4   Lng         5 non-null      float64
 5   Max_Temp    5 non-null      float64
 6   Humidity    5 non-null      float64
 7   Cloudiness  5 non-null      float64
 8   Wind_Speed  5 non-null      float64
 9   Country     5 non-null      object 
 10  Date        5 non-null      int64  
dtypes: float64(6), int64(3), object(2)
memory usage: 480.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Create a new dataframe
hotel_df = pd.DataFrame({'City':pd.Series([], dtype='str'),
                        'Country':pd.Series([], dtype='str'),
                        'Lat':pd.Series([], dtype='float'),
                        'Lng':pd.Series([], dtype='float'),
                        'Hotel_Name':pd.Series([], dtype='str'),
                        'Hotel_Ratings':pd.Series([], dtype='str')})                   
print (hotel_df)


Empty DataFrame
Columns: [City, Country, Lat, Lng, Hotel_Name, Hotel_Ratings]
Index: []


In [16]:
# Build URL using the Google Maps API
    
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
for i, row in weather_df[:1].iterrows():
    
    # row[9] is pointing to values in column # 9 
    
    city = row[2]
    country = row[9]    
    #print(f"{country}")
    
    lat_lng = weather_df[["Lat", "Lng"]].iloc[i]
    #print(lat_lng)
    
    lat = lat_lng[0]
    lng = lat_lng[1]
    
    location = f'{lat},{lng}'
    
    target_search = "Hotel"
    hotel_coords = location
    radius = "5000"
    
    # redefine params
    params = {
        "location": hotel_coords,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
        }
    
    # Run request
    response = requests.get(base_url, params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
    
        latitutde = response["results"][0]["geometry"]["location"]["lat"]
        longitude = response["results"][0]["geometry"]["location"]["lng"]
        name = response["results"][0]["name"]
        rating = response["results"][0]["rating"]
                      
        hotel_df.loc[i, "Lat"] = latitutde
        hotel_df.loc[i, "Lng"] = longitude
        hotel_df.loc[i, "Hotel_Name"] = name
        hotel_df.loc[i, "Hotel_Ratings"] = rating
        hotel_df.loc[i, "City"] = city
        hotel_df.loc[i, "Country"] = country
        
        time.sleep(1)
       
    except (KeyError, IndexError):   
        
        print("Hotel not found!")

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ7LVZo519X0EU9go0EDdw1LSA2QejDg4GQByHy2gncJQj-te3L69qgaQwFy2GoYsqYqdjquNywGYbx0LUKHXxKH-Evb42T08f0Ul4xjzMUb7q15sO6zIHHsowZ9OJ5eM-i4IWfr5U5lLruayuZJCJkx6Pt9bp-BF72e2yPjYd9r3rJw488rWE3g6EDefkEAJghBg57xAtgjRycX_UoZllALauE8qrLiv0nZfaz8pak7vuBl7wDJfjW9_tef15Fm9QxE5i54KtN6EVTZFESpsKs9v3O6oskSKiQOHqfnW2FYbG_aIRrDwpENCXyxjDkcjKNCm5_GJSkVfEsqxjdBoaLbOcqqQjhY8Cbr7LJmMZwtx7XJgbtTOux9ipLxOZqk5MFpc4UD6kntUEPrE5ixROGX9JeHXR21P-Eva5xYIIgPOfmdloGXg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.9579372,
                    "lng": 115.8647079
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.95655822010728,
                        "lng": 115.8663108298928
                    },
                    "southwest": {
                        "lat": -31.95925787989272,

In [ ]:
hotel_df.head()

In [ ]:
hotel_df.info()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Convert 'Hotel_Name' as a list

hotel_name = hotel_df["Hotel_Name"].tolist()

In [ ]:
hotel_df[['Lat', 'Lng']]

In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the hotel name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel name: {name}" for name in hotel_name])
fig.add_layer(markers)
fig

In [ ]:
# Convert 'City' name as a list
city_name = hotel_df["City"].tolist()



In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the City name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"City name: {name}" for name in city_name])
fig.add_layer(markers)
fig

In [ ]:
# Convert 'Country' name as a list
country_name = hotel_df["Country"].tolist()

In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the Country name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Country name: {name}" for name in country_name])
fig.add_layer(markers)
fig